In [36]:
import time
import numpy as np
import pandas as pd
import time
import pickle
import sys
from tqdm import tqdm
from selenium import webdriver

In [8]:
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(3)
driver.get('https:/1xbet.whoscored.com/')
driver.close()

In [46]:
def replace_pd(df):
    mapping = {'-': 0} 
    
    replace_dict = {}
    
    for colum in df.columns:
        replace_dict[colum] = mapping
    return df.replace(replace_dict)

In [17]:
def crawling_league_teams(team_id, api_delay_term=5):

    """
    cawling league team_id and team name datas
    
    Arg :
        team_id : one of you want league team_id & parameter data type int or str
    
    return :
        crawling league team_id, team_name datas belong team_id parameter
        return pandas dataframe columns=team_id, team_name
    

    """
    
    # connect webdriver
    url = "https://www.whoscored.com/Teams/" + str(team_id)
    driver = webdriver.Chrome('C:/Users/Infosci_center/Desktop/soccer/chromedriver')
    driver.get(url)

    
    # wait get league team datas
    time.sleep(api_delay_term) 
    
    # make pandas dataframe
    team_df = pd.DataFrame(columns=["team_id","team_name"])
    
    # get team datas
    teams = driver.find_elements_by_css_selector("#teams option")
    for team in teams:
        team_name = team.text
        team_id = team.get_attribute("value").split("/")[2]
        team_df.loc[len(team_df)] = {"team_id":team_id, "team_name":team_name }
        
    # close webdriver
    driver.close()
    
    return replace_pd(team_df)

In [18]:
PL = crawling_league_teams(26)

In [19]:
PL

,team_id,team_name
0,13,Arsenal
1,24,Aston Villa
2,211,Brighton
3,184,Burnley
4,15,Chelsea
5,162,Crystal Palace
6,31,Everton
7,170,Fulham
8,19,Leeds
9,14,Leicester


In [60]:
def crawling_player_summary(team_id, api_delay_term=5):
    """
    crawling player statistics of certain team
    
    Args :
        team_id : team number 
        
    return :
        player statistics (dataframe)
    
    """

    # connect webdriver
    url = "https://www.whoscored.com/Teams/" + str(team_id)
    driver = webdriver.Chrome('C:/Users/Infosci_center/Desktop/soccer/chromedriver')
    driver.get(url)

    # wait for getting data
    time.sleep(api_delay_term)
    
    # make pandas dataframe
    player_summary_df = pd.DataFrame(columns=[
        "player_number", "name","age","position","tall","weight","games",
        "mins", "goals", "asists", "yel", "red", "spg", "ps","aw","motm", "rating"
        ])
    
    # get player summay datas
    elements = driver.find_elements_by_css_selector("#player-table-statistics-body tr")
    for element in elements:

        player_dict = { 
            "player_number": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector("a")[0].get_attribute("href").split("/")[4], 
            "name": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector("a")[0].find_elements_by_css_selector("span")[0].text,     
            "age": element.find_elements_by_css_selector("td")[1].find_elements_by_css_selector("span")[0].text,
            "position": element.find_elements_by_css_selector("td")[1].find_elements_by_css_selector("span")[1].text[1:], 
            "tall": element.find_elements_by_css_selector("td")[2].text,
            "weight": element.find_elements_by_css_selector("td")[3].text, 
            "games": element.find_elements_by_css_selector("td")[4].text, 
            "mins": element.find_elements_by_css_selector("td")[5].text,
            "goals": element.find_elements_by_css_selector("td")[6].text,
            "asists": element.find_elements_by_css_selector("td")[7].text,
            "yel": element.find_elements_by_css_selector("td")[8].text,
            "red": element.find_elements_by_css_selector("td")[9].text,
            "spg": element.find_elements_by_css_selector("td")[10].text,
            "ps": element.find_elements_by_css_selector("td")[11].text,
            "aw": element.find_elements_by_css_selector("td")[12].text,
            "motm": element.find_elements_by_css_selector("td")[13].text,
            "rating": element.find_elements_by_css_selector("td")[14].text,
        }
        
        player_summary_df.loc[len(player_summary_df)] = player_dict
    
    # close webdriver
    driver.close()
    
    return player_summary_df

In [61]:
liverpool = crawling_player_summary(26, api_delay_term=5)

In [64]:
liverpool = replace_pd(liverpool)

In [65]:
liverpool

,player_number,name,age,position,tall,weight,games,mins,goals,asists,yel,red,spg,ps,aw,motm,rating
0,108226,Mohamed Salah,,,175,71,1,90,3,0,0,0,9,80.4,0,1,9.88
1,109915,Sadio Mané,,,175,69,1,90,0,0,0,0,5,81.1,1,0,7.52
2,115726,Andrew Robertson,,,178,64,1,90,0,1,0,0,0,62.8,2,0,7.00
3,96182,Roberto Firmino,,,181,76,1,90,0,0,1,0,2,85,0,0,6.89
4,33568,Georginio Wijnaldum,,,175,69,1,90,0,0,0,0,2,81,0,0,6.86
5,68659,Jordan Henderson,,,182,67,1,66,0,0,0,0,1,76.3,0,0,6.86
6,136451,Joe Gomez,,,188,77,1,90,0,0,0,0,0,79.2,4,0,6.77
7,243510,Naby Keita,,,172,64,1,59,0,0,0,0,1,81.5,1,0,6.74
8,115916,Fabinho,,,188,78,0(1),31,0,0,0,0,0,84.6,1,0,6.69
9,318871,Trent Alexander-Arnold,,,175,69,1,89,0,0,0,0,0,67.6,1,0,6.34


In [9]:
# Premier league 1920 plyaer stat table(URL)
URL = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/7811/Stages/17590/PlayerStatistics/England-Premier-League-2019-2020#stage-top-player-stats-detailed'

In [16]:
url = str(URL)
driver = webdriver.Chrome('./chromedriver')
driver.get(url)
    
# time.sleep(5)

# driver.find_element_by_id('next').click()

# time.sleep(5)

# driver.find_element_by_id('next').click()

In [41]:
def crawling_player_statistics(URL):
    """
    crawling player statistics of certain league and season.
    
    Args :
        URL : player statistics webpage URL
        
    return :
        player statistics (dataframe)
    
    """
    url = str(URL)
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    
    time.sleep(3)
    
    page = driver.find_elements_by_css_selector('#statistics-paging-summary > div > dl.listbox.right > dt > b')[0].text
    page = int(page.split('/')[1].split(' ')[0])
    player_statistics_df = pd.DataFrame(columns=[
        "player_name", "team_number", "team_name", "Apps", "Mins", "Goals",
        "Assists", "Yel", "Red", "SpG","PS%", "AerialsWon","MoM","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
        for i in range(page):
            if i != 0:
                driver.find_element_by_id('next').click()
                time.sleep(3)

            temp_df = pd.DataFrame(columns=[
                "player_name", "team_number", "team_name", "Apps", "Mins", "Goals",
                "Assists", "Yel", "Red", "SpG","PS%", "AerialsWon","MoM","Rating"])
            elements = driver.find_elements_by_css_selector("#player-table-statistics-body tr")

            for element in elements:
                player_statistics_dict = { 
                    "player_name": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector("a")[0].
                    find_elements_by_css_selector("span")[0].text,
                    "team_number": element.find_elements_by_css_selector("td")[0].
                    find_elements_by_css_selector("a")[1].get_attribute("href").split("/")[4],
                    "team_name": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector
                    ("a")[1].find_elements_by_css_selector("span")[0].text,
                    "Apps": element.find_elements_by_css_selector("td")[2].text,
                    "Mins": element.find_elements_by_css_selector("td")[3].text, 
                    "Goals": element.find_elements_by_css_selector("td")[4].text,
                    "Assists": element.find_elements_by_css_selector("td")[5].text, 
                    "Yel": element.find_elements_by_css_selector("td")[6].text, 
                    "Red": element.find_elements_by_css_selector("td")[7].text,
                    "SpG": element.find_elements_by_css_selector("td")[8].text,
                    "PS%": element.find_elements_by_css_selector("td")[9].text,
                    "AerialsWon": element.find_elements_by_css_selector("td")[10].text,
                    "MoM": element.find_elements_by_css_selector("td")[11].text,
                    "Rating": element.find_elements_by_css_selector("td")[12].text,
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict

            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
            
    driver.close()
    
    return(player_statistics_df)
    

In [42]:
PL_player_statistics_1920 = crawling_player_statistics(URL)

page 27: 100%|█████████████████████████████████████████████████████████████████████████| 28/28 [02:24<00:00,  5.18s/it]


In [43]:
PL_player_statistics_1920

,player_name,team_number,team_name,Apps,Mins,Goals,Assists,Yel,Red,SpG,PS%,AerialsWon,MoM,Rating
0,Kevin De Bruyne,167,"Man City,",32(3),2800,13,20,3,-,2.8,81.5,0.5,10,7.97
1,Ricardo Pereira,14,"Leicester,",28,2520,3,2,1,-,0.6,78.9,1.5,2,7.50
2,Adama Traoré,161,"Wolves,",27(10),2608,4,9,1,-,1.2,74.4,1.2,6,7.49
3,Riyad Mahrez,167,"Man City,",21(12),1942,11,9,-,-,2.2,90.1,0.3,3,7.48
4,Sadio Mané,26,"Liverpool,",31(4),2756,18,7,3,-,2.2,81.6,1.2,7,7.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,Glenn Murray,211,"Brighton,",7(16),790,1,1,-,-,0.8,61.7,1.7,-,6.25
2,Marco Stiepermann,168,"Norwich,",14(10),1189,-,-,6,-,0.8,71.5,0.6,-,6.22
3,Mario Vrancic,168,"Norwich,",6(14),630,1,-,3,-,0.7,78.3,0.4,-,6.17
4,Andre Gray,27,"Watford,",7(16),747,2,-,1,-,0.8,65.3,0.7,-,6.15


In [44]:
def replace_Apps(df):
    
    sub = list()
    for i in list(df['Apps']):
        if '(' in i :
            sub.append(i.split('(')[1].split(')')[0])
        else :
            sub.append('0')
    df['sub'] = sub
    df['Apps'] = list(map(lambda x: x.split('(')[0], list(df['Apps'])))
    df['team_name'] = list(map(lambda x: x.split(',')[0], list(df['team_name'])))
    return(df)          

In [47]:
PL_player_summary = replace_pd(replace_Apps(PL_player_statistics_1920))

In [48]:
#PL_player_summary.to_csv('PL_player_summary_1920.csv')

In [59]:
 def crawling_player_statistics_defensive(URL):
    """
    crawling player statistics(defensive) of certain league and season.
    
    Args :
        URL : player statistics webpage URL
        
    return :
        player statistics (dataframe)
    
    """
    url = str(URL)
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    
    time.sleep(4)

    driver.find_element_by_id('next').click()

    time.sleep(4)

    driver.find_element_by_id('first').click()
    
    time.sleep(4)
    
    driver.find_element_by_partial_link_text("Defensive").click()
    
    time.sleep(4)
    page = driver.find_elements_by_css_selector('#statistics-paging-defensive > div > dl.listbox.right > dt > b')[0].text
    page = int(page.split('/')[1].split(' ')[0])
    
    player_statistics_df = pd.DataFrame(columns=[
            "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Tackles",
            "Inter", "Fouls", "Offsides", "Clear","Drb", "Blocks","OwnG","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
        for i in range(page):
            if i != 0:
                driver.find_element_by_partial_link_text('next').click()
                time.sleep(4)

            temp_df = pd.DataFrame(columns=[
                "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Tackles",
                "Inter", "Fouls", "Offsides", "Clear","Drb", "Blocks","OwnG","Rating"])

            elements = driver.find_elements_by_css_selector("#player-table-statistics-body tr")[10:]

            for element in elements:
                player_statistics_dict = { 
                    "player_name": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector("a")[0].
                    find_elements_by_css_selector("span")[0].text,
                    "team_number": element.find_elements_by_css_selector("td")[0].
                    find_elements_by_css_selector("a")[1].get_attribute("href").split("/")[4],
                    "team_name": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector
                    ("a")[1].find_elements_by_css_selector("span")[0].text,
                    "age": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector
                    ("span")[4].text,
                    "position": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector
                    ("span")[5].text,
                    "Apps": element.find_elements_by_css_selector("td")[2].text,
                    "Mins": element.find_elements_by_css_selector("td")[3].text, 
                    "Tackles": element.find_elements_by_css_selector("td")[4].text,
                    "Inter": element.find_elements_by_css_selector("td")[5].text, 
                    "Fouls": element.find_elements_by_css_selector("td")[6].text, 
                    "Offsides": element.find_elements_by_css_selector("td")[7].text,
                    "Clear": element.find_elements_by_css_selector("td")[8].text,
                    "Drb": element.find_elements_by_css_selector("td")[9].text,
                    "Blocks": element.find_elements_by_css_selector("td")[10].text,
                    "OwnG": element.find_elements_by_css_selector("td")[11].text,
                    "Rating": element.find_elements_by_css_selector("td")[12].text,
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict


            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
        
    driver.close()
    
    return(player_statistics_df)
    

In [60]:
PL_player_defensive_1920 = crawling_player_statistics_defensive(URL)

page 28: 100%|█████████████████████████████████████████████████████████████████████████| 28/28 [03:01<00:00,  6.49s/it]


In [61]:
replace_pd(replace_Apps(PL_player_defensive_1920)).to_csv('PL_player_defensive_1920.csv')

In [62]:
replace_pd(replace_Apps(PL_player_defensive_1920)).head()

,player_name,team_number,team_name,age,position,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating,sub
0,Kevin De Bruyne,167,Man City,30,", M(CLR),FW",32,2800,1.3,0.5,0.7,0,0.7,1.4,0.1,0,7.97,0
1,Ricardo Pereira,14,Leicester,28,", D(LR),M(R)",28,2520,4.3,1.8,1.6,0.3,2.6,1.9,0.2,0,7.50,0
2,Adama Traoré,161,Wolves,25,", M(CLR),FW",27,2608,0.9,0.4,1.1,0,0.4,0.4,0.1,0,7.49,0
3,Riyad Mahrez,167,Man City,30,", AM(CLR)",21,1942,0.8,0.5,0.4,0,0.2,0.8,0,0,7.48,0
4,Sadio Mané,26,Liverpool,29,", AM(CLR),FW",31,2756,1.3,0.4,1.3,0,0.2,0.8,0,0,7.45,0


In [65]:
def crawling_player_statistics_offensive(URL):
    """
    crawling player statistics(offensive) of certain league and season.
    
    Args :
        URL : player statistics webpage URL
        
    return :
        player statistics (dataframe)
    
    """
    url = str(URL)
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    
    time.sleep(4)

    driver.find_element_by_id('next').click()

    time.sleep(4)

    driver.find_element_by_id('first').click()
    
    time.sleep(4)
    
    driver.find_element_by_partial_link_text("Offensive").click()
    
    time.sleep(4)
    page = driver.find_elements_by_css_selector('#statistics-paging-offensive > div > dl.listbox.right > dt > b')[0].text
    page = int(page.split('/')[1].split(' ')[0])
    
    player_statistics_df = pd.DataFrame(columns=[
            "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Goals",
            "Assists", "SpG", "KeyP", "Dribble","Fouled", "Off","Disp","UnsTch","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
        for i in range(page):
            if i != 0:
                driver.find_element_by_partial_link_text('next').click()
                time.sleep(5)

            temp_df = pd.DataFrame(columns=[
                "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Goals",
                "Assists", "SpG", "KeyP", "Dribble","Fouled", "Off","Disp","UnsTch","Rating"])

            elements = driver.find_elements_by_css_selector("#player-table-statistics-body tr")[10:]

            for element in elements:
                player_statistics_dict = { 
                    "player_name": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector("a")[0].
                    find_elements_by_css_selector("span")[0].text,
                    "team_number": element.find_elements_by_css_selector("td")[0].
                    find_elements_by_css_selector("a")[1].get_attribute("href").split("/")[4],
                    "team_name": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector
                    ("a")[1].find_elements_by_css_selector("span")[0].text,
                    "age": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector
                    ("span")[4].text,
                    "position": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector
                    ("span")[5].text,
                    "Apps": element.find_elements_by_css_selector("td")[2].text,
                    "Mins": element.find_elements_by_css_selector("td")[3].text, 
                    "Goals": element.find_elements_by_css_selector("td")[4].text,
                    "Assists": element.find_elements_by_css_selector("td")[5].text, 
                    "SpG": element.find_elements_by_css_selector("td")[6].text, 
                    "KeyP": element.find_elements_by_css_selector("td")[7].text,
                    "Dribble": element.find_elements_by_css_selector("td")[8].text,
                    "Fouled": element.find_elements_by_css_selector("td")[9].text,
                    "Off": element.find_elements_by_css_selector("td")[10].text,
                    "Disp": element.find_elements_by_css_selector("td")[11].text,
                    "UnsTch": element.find_elements_by_css_selector("td")[12].text,
                    "Rating": element.find_elements_by_css_selector("td")[13].text,
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict


            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
        
    driver.close()
    
    return(player_statistics_df)

In [66]:
PL_player_offensive_1920 = crawling_player_statistics_offensive(URL)

page 28: 100%|█████████████████████████████████████████████████████████████████████████| 28/28 [03:35<00:00,  7.69s/it]


In [68]:
PL_player_offensive_1920.head()

,player_name,team_number,team_name,age,position,Apps,Mins,Goals,Assists,SpG,KeyP,Dribble,Fouled,Off,Disp,UnsTch,Rating
0,Kevin De Bruyne,167,"Man City,",30,", M(CLR),FW",32(3),2800,13,20,2.8,3.9,1.4,0.8,0.1,0.9,1.3,7.97
1,Ricardo Pereira,14,"Leicester,",28,", D(LR),M(R)",28,2520,3,2,0.6,1,2.1,1.3,-,1.1,1.6,7.50
2,Adama Traoré,161,"Wolves,",25,", M(CLR),FW",27(10),2608,4,9,1.2,1.3,5,2.1,0.2,1.8,2.4,7.49
3,Riyad Mahrez,167,"Man City,",30,", AM(CLR)",21(12),1942,11,9,2.2,1.8,1.6,0.9,0.4,1,1.4,7.48
4,Sadio Mané,26,"Liverpool,",29,", AM(CLR),FW",31(4),2756,18,7,2.2,1.7,2,1.5,0.4,1.6,2.9,7.45


In [69]:
replace_pd(replace_Apps(PL_player_offensive_1920)).to_csv('PL_player_offensive_1920.csv')

In [72]:
def crawling_player_statistics_passing(URL):
    url = str(URL)
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    
    time.sleep(4)

    driver.find_element_by_id('next').click()

    time.sleep(4)

    driver.find_element_by_id('first').click()
    
    time.sleep(4)
    
    driver.find_element_by_partial_link_text("Passing").click()
    
    time.sleep(4)
    page = driver.find_elements_by_css_selector('#statistics-paging-passing > div > dl.listbox.right > dt > b')[0].text
    page = int(page.split('/')[1].split(' ')[0])
    
    player_statistics_df = pd.DataFrame(columns=[
            "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Assists",
            "KeyP", "AvgP","PS%", "Crosses","LongB","ThrB","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
        for i in range(page):
            if i != 0:
                driver.find_element_by_partial_link_text('next').click()
                time.sleep(5)

            temp_df = pd.DataFrame(columns=[
                "player_name", "team_number", "team_name", "age","position", "Apps", "Mins","Assists",
                "KeyP", "AvgP","PS%", "Crosses","LongB","ThrB","Rating"])

            elements = driver.find_elements_by_css_selector("#player-table-statistics-body tr")[10:]

            for element in elements:
                player_statistics_dict = { 
                    "player_name": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector("a")[0].
                    find_elements_by_css_selector("span")[0].text,
                    "team_number": element.find_elements_by_css_selector("td")[0].
                    find_elements_by_css_selector("a")[1].get_attribute("href").split("/")[4],
                    "team_name": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector
                    ("a")[1].find_elements_by_css_selector("span")[0].text,
                    "age": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector
                    ("span")[4].text,
                    "position": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector
                    ("span")[5].text,
                    "Apps": element.find_elements_by_css_selector("td")[2].text,
                    "Mins": element.find_elements_by_css_selector("td")[3].text, 
                    "Assists": element.find_elements_by_css_selector("td")[4].text,
                    "KeyP": element.find_elements_by_css_selector("td")[5].text, 
                    "AvgP": element.find_elements_by_css_selector("td")[6].text, 
                    "PS%": element.find_elements_by_css_selector("td")[7].text,
                    "Crosses": element.find_elements_by_css_selector("td")[8].text,
                    "LongB": element.find_elements_by_css_selector("td")[9].text,
                    "ThrB": element.find_elements_by_css_selector("td")[10].text,
                    "Rating": element.find_elements_by_css_selector("td")[11].text,
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict


            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
        
    driver.close()
    
    return(player_statistics_df)

In [73]:
PL_player_passing_1920 = crawling_player_statistics_passing(URL)

page 28: 100%|█████████████████████████████████████████████████████████████████████████| 28/28 [03:23<00:00,  7.27s/it]


In [74]:
PL_player_passing_1920.head()

,player_name,team_number,team_name,age,position,Apps,Mins,Assists,KeyP,AvgP,PS%,Crosses,LongB,ThrB,Rating
0,Kevin De Bruyne,167,"Man City,",30,", M(CLR),FW",32(3),2800,20,3.9,54.5,81.5,2.1,3.5,0.4,7.97
1,Ricardo Pereira,14,"Leicester,",28,", D(LR),M(R)",28,2520,2,1,56.1,78.9,0.3,2.1,0.1,7.50
2,Adama Traoré,161,"Wolves,",25,", M(CLR),FW",27(10),2608,9,1.3,19.2,74.4,1.2,0.2,0.1,7.49
3,Riyad Mahrez,167,"Man City,",30,", AM(CLR)",21(12),1942,9,1.8,33.2,90.1,0.6,1.7,0.2,7.48
4,Sadio Mané,26,"Liverpool,",29,", AM(CLR),FW",31(4),2756,7,1.7,31.5,81.6,0.3,1,0.1,7.45


In [75]:
replace_pd(replace_Apps(PL_player_passing_1920)).to_csv('PL_player_passing_1920.csv')

In [76]:
passing=pd.read_csv('PL_player_passing_1920.csv')
summary=pd.read_csv('PL_player_summary_1920.csv')
offensive=pd.read_csv('PL_player_offensive_1920.csv')
defensive=pd.read_csv('PL_player_defensive_1920.csv')

In [77]:
PL_player_1920 = pd.merge(pd.merge(passing,summary), pd.merge(offensive,defensive))
PL_player_1920['position'] = list(map(lambda x: x.split(' ')[1],list(PL_player_1920['position'])))

In [78]:
PL_player_1920.head()

,Unnamed: 0,player_name,team_number,team_name,age,position,Apps,Mins,Assists,KeyP,...,Disp,UnsTch,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG
0,0,Kevin De Bruyne,167,Man City,30,"M(CLR),FW",32,2800,20,3.9,...,0.9,1.3,1.3,0.5,0.7,0.0,0.7,1.4,0.1,0
1,1,Ricardo Pereira,14,Leicester,28,"D(LR),M(R)",28,2520,2,1.0,...,1.1,1.6,4.3,1.8,1.6,0.3,2.6,1.9,0.2,0
2,2,Adama Traoré,161,Wolves,25,"M(CLR),FW",27,2608,9,1.3,...,1.8,2.4,0.9,0.4,1.1,0.0,0.4,0.4,0.1,0
3,3,Riyad Mahrez,167,Man City,30,AM(CLR),21,1942,9,1.8,...,1.0,1.4,0.8,0.5,0.4,0.0,0.2,0.8,0.0,0
4,4,Sadio Mané,26,Liverpool,29,"AM(CLR),FW",31,2756,7,1.7,...,1.6,2.9,1.3,0.4,1.3,0.0,0.2,0.8,0.0,0


In [79]:
PL_player_1920.iloc[:,1:].to_csv('PL_player_1920.csv')